# Imports

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Conv2D, MaxPooling2D, Conv1D, Flatten
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization

# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed
from keras.layers.core import Dense, Dropout

Using TensorFlow backend.


# Data preprocessing

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [6]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [7]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [8]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [9]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [10]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [11]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [12]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [13]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [14]:
# Initializing parameters
epochs = 30
batch_size = 30
# n_hidden = 32

In [15]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

# The DNN-based Fusion Model

- https://arxiv.org/pdf/1805.07020.pdf
- https://www.researchgate.net/publication/305649713_Deep_learning_for_human_activity_recognition_A_resource_efficient_implementation_on_low-power_devices

In [100]:
# https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/
# https://machinelearningmastery.com/deep-learning-models-for-human-activity-recognition/
# https://github.com/UdiBhaskar/Human-Activity-Recognition--Using-Deep-NN/blob/master/Human%20Activity%20Detection-Without%20Verbose%20.ipynb

### Understanding and Improving Deep Neural Network for Activity Recognition

In [89]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from keras.models import load_model

### model wise data preparation

#### Train and Test data for model-1

In [81]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

#### Train and Test data for model-2

In [82]:
# selecting axis specific data
X_train_1 = X_train[:,:,[1,2,3,4,5,6]]
X_test_1 = X_test[:,:,[1,2,3,4,5,6]]

#### Train and Test data for model-3

In [83]:
# selecting axis specific data
X_train_2 = X_train[:,:,[1,2,3,4]]
X_test_2 = X_test[:,:,[1,2,3,4]]

In [84]:
# X_train_2 = X_train[:,:,[5,6,7,8]]
# X_test_2 = X_test[:,:,[5,6,7,8]]

In [85]:
# # selecting axis specific data
# X_train_1 = X_train[:,:,[3,4,5,6,7,8]]
# X_train_2 = X_train[:,:,[3,4,6,7]]

In [86]:
# # selecting axis specific data
# X_test_1 = X_test[:,:,[3,4,5,6,7,8]]
# X_test_2 = X_test[:,:,[3,4,6,7]]

In [88]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
print(timesteps)
print(input_dim)
print(len(X_train))
num_classes = 6

128
9
7352


#### Dataset for model-1

In [91]:
# Test and Train data reshaping for 1st dataset
# input image dimensions

img_rows, img_cols = 128, 9
# an activity is governed by sequence of activities. 8 sequences informations are given to CNN model which will be later given to LSTM unit as a sequence information.
# 7352 = 919*8
X_train = X_train.reshape(919,8,128,9,1)
Y_train = Y_train.reshape(919,8,6)

# removing last 3 data pointjust to make test data sequence compatible
# 2944 = 368*8
X_test = X_test[:-3]
Y_test = Y_test[:-3]

X_test = X_test.reshape(368,8,128,9,1)
Y_test = Y_test.reshape(368,8,6)

# Input shape for model-1
input_shape_1 = ( X_train.shape[1], X_train.shape[2], X_train.shape[3], X_train.shape[4])
print(input_shape_1)

(8, 128, 9, 1)


In [92]:
X_train.shape, X_test.shape

((919, 8, 128, 9, 1), (368, 8, 128, 9, 1))

#### Dataset for model-2

In [93]:
# Test and Train data reshaping for 1st dataset
# input image dimensions

img_rows, img_cols = 128, 6
# an activity is governed by sequence of activities. 8 sequences informations are given to CNN model which will be later given to LSTM unit as a sequence information.
# 7352 = 919*8
X_train_1 = X_train_1.reshape(919,8,128,6,1)

# removing last 3 data pointjust to make test data sequence compatible
# 2944 = 368*8
X_test_1 = X_test_1[:-3]
X_test_1 = X_test_1.reshape(368,8,128,6,1)

# Input shape for model-2
input_shape_2 = (X_train_1.shape[1], X_train_1.shape[2], X_train_1.shape[3], X_train_1.shape[4])
print(input_shape_2)

(8, 128, 6, 1)


In [94]:
X_train_1.shape, X_test_1.shape

((919, 8, 128, 6, 1), (368, 8, 128, 6, 1))

#### Dataset for model-3

In [95]:
# Test and Train data reshaping for 1st dataset
# input image dimensions

img_rows, img_cols = 128, 4
# an activity is governed by sequence of activities. 8 sequences are given to CNN model which will be later given to LSTM unit as a sequence information.
# 7352 = 919*8
X_train_2 = X_train_2.reshape(919,8,128,4,1)

# removing last 3 data pointjust to make test data sequence compatible
# 2944 = 368*8
X_test_2 = X_test_2[:-3]
X_test_2 = X_test_2.reshape(368,8,128,4,1)

# Input shape for model-2
input_shape_3 = (X_train_2.shape[1], X_train_2.shape[2], X_train_2.shape[3], X_train_2.shape[4])
print(input_shape_3)

(8, 128, 4, 1)


In [96]:
X_train_2.shape, X_test_2.shape

((919, 8, 128, 4, 1), (368, 8, 128, 4, 1))

In [97]:
def get_model(shape_cnn, shape_lstm):
    # print(shape_cnn, shape_lstm)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(128, kernel_size=(5,1),  activation='relu', input_shape= shape_cnn)))
    model.add(TimeDistributed(Conv2D(64, (5, 1), activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Conv2D(32, (5, 1), activation='relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(TimeDistributed(Dense(32, activation='relu')))
    model.add(LSTM(units=64, return_sequences=True, input_shape = shape_lstm))
    model.add(Dense(num_classes, activation='softmax'))
    # compiling the model
    # model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Model-1 Training

In [30]:
model_1 = get_model(input_shape_1,(input_shape_1[1],input_shape_1[2]))
model_1.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
919/919 [==============================] - 83s 91ms/step - loss: 0.7057 - accuracy: 0.6942
Epoch 2/30
919/919 [==============================] - 84s 91ms/step - loss: 0.4070 - accuracy: 0.8470
Epoch 3/30
919/919 [==============================] - 82s 89ms/step - loss: 0.2889 - accuracy: 0.9007
Epoch 4/30
919/919 [==============================] - 82s 89ms/step - loss: 0.2282 - accuracy: 0.9233
Epoch 5/30
919/919 [==============================] - 82s 89ms/step - loss: 0.1990 - accuracy: 0.9329
Epoch 6/30
919/919 [==============================] - 82s 90ms/step - loss: 0.2003 - accuracy: 0.9329
Epoch 7/30
919/919 [==============================] - 83s 91ms/step - loss: 0.1572 - accuracy: 0.9524
Epoch 8/30
919/919 [==============================] - 83s 90ms/step - loss: 0.1294 - accuracy: 0.9580
Epoch 9/30
919/919 [==============================] - 90s 98ms/step - loss: 0.1172 - accu

In [31]:
model_1.save('CNN_LSTM_Model_1.h5')

In [32]:
score = model_1.evaluate(X_test, Y_test)

368/368 [==============================] - 7s 20ms/step


In [33]:
score

[0.22175494484279468, 0.9225543737411499]

### Model-2 Training

In [34]:
model_2 = get_model(input_shape_2,(input_shape_2[1],input_shape_2[2]))
model_2.fit(X_train_1, Y_train, batch_size=batch_size, epochs=epochs, verbose=1)
model_2.save('CNN_LSTM_Model_2.h5')

Epoch 1/30
919/919 [==============================] - 60s 65ms/step - loss: 1.1910 - accuracy: 0.4777
Epoch 2/30
919/919 [==============================] - 58s 63ms/step - loss: 0.6051 - accuracy: 0.7135
Epoch 3/30
919/919 [==============================] - 56s 61ms/step - loss: 0.4745 - accuracy: 0.7671
Epoch 4/30
919/919 [==============================] - 54s 59ms/step - loss: 0.4129 - accuracy: 0.7916
Epoch 5/30
919/919 [==============================] - 62s 68ms/step - loss: 0.3534 - accuracy: 0.8301
Epoch 6/30
919/919 [==============================] - 60s 65ms/step - loss: 0.3053 - accuracy: 0.8546
Epoch 7/30
919/919 [==============================] - 54s 59ms/step - loss: 0.3044 - accuracy: 0.8523
Epoch 8/30
919/919 [==============================] - 54s 58ms/step - loss: 0.2536 - accuracy: 0.8761
Epoch 9/30
919/919 [==============================] - 53s 58ms/step - loss: 0.2234 - accuracy: 0.9059
Epoch 10/30
919/919 [==============================] - 53s 58ms/step - loss: 0.206

In [35]:
score = model_2.evaluate(X_test_1, Y_test)

368/368 [==============================] - 5s 14ms/step


In [36]:
score

[0.1947319541612397, 0.9290081262588501]

### Model-3 Training


In [98]:
model_3 = get_model(input_shape_3,(input_shape_3[1],input_shape_3[2]))
model_3.fit(X_train_2, Y_train, batch_size=batch_size, epochs=epochs, verbose=1)
model_3.save('CNN_LSTM_Model_3.h5')

Epoch 1/30
919/919 [==============================] - 74s 81ms/step - loss: 0.9507 - accuracy: 0.6129
Epoch 2/30
919/919 [==============================] - 73s 79ms/step - loss: 0.6082 - accuracy: 0.7337
Epoch 3/30
919/919 [==============================] - 50s 55ms/step - loss: 0.5404 - accuracy: 0.7558
Epoch 4/30
919/919 [==============================] - 37s 40ms/step - loss: 0.4534 - accuracy: 0.8013
Epoch 5/30
919/919 [==============================] - 37s 40ms/step - loss: 0.4212 - accuracy: 0.8184
Epoch 6/30
919/919 [==============================] - 36s 39ms/step - loss: 0.3842 - accuracy: 0.8387
Epoch 7/30
919/919 [==============================] - 37s 40ms/step - loss: 0.3982 - accuracy: 0.8335
Epoch 8/30
919/919 [==============================] - 36s 39ms/step - loss: 0.3565 - accuracy: 0.8589
Epoch 9/30
919/919 [==============================] - 39s 43ms/step - loss: 0.3052 - accuracy: 0.8974
Epoch 10/30
919/919 [==============================] - 49s 54ms/step - loss: 0.303

In [101]:
score = model_3.evaluate(X_test_2, Y_test)

368/368 [==============================] - 3s 9ms/step


In [74]:
score

[0.23070418964261594, 0.9161005616188049]

### Voting mechanism for the final classification

In [75]:
y_test = Y_test.reshape(2944,6)

y1 = model_1.predict_proba(X_test)
y1 = y1.reshape(2944,6)

y2 = model_2.predict_proba(X_test_1)
y2 = y2.reshape(2944,6)

y3 = model_3.predict_proba(X_test_2)
y3 = y3.reshape(2944,6)

In [76]:
y_pred = []
for i in range(len(y1)):
    tmp = [0,0,0,0,0,0]
    
    op1 = np.amax(y1[i])
    index1 = y1[i].argmax(axis=0)
    tmp[index1] = 1
    
    op2 = np.amax(y2[i])
    index2 = y2[i].argmax(axis=0)
    tmp[index2] = 1
    
    op3 = np.amax(y3[i])
    index3 = y3[i].argmax(axis=0)
    tmp[index3] = 1
    
    l = [op1, op2, op3]
    if sum(tmp)==1:
        y_pred.append(tmp)
    
    else:
        tmp = [0,0,0,0,0,0]
        ind = l.index(max(l))
        if ind == 0:
            tmp[index1] = 1
            y_pred.append(tmp)
        
        elif ind ==1:
            tmp[index2] = 1
            y_pred.append(tmp)
        
        elif ind ==2:
            tmp[index3] = 1
            y_pred.append(tmp)
   
    

In [78]:
# Confusion Matrix
y_pred = np.array(y_pred)
print(confusion_matrix(y_test, y_pred))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      446        43        0                   0   
STANDING                 0       19       507        1                   0   
WALKING                  1        0         0      479                  15   
WALKING_DOWNSTAIRS       0        0         0       21                 389   
WALKING_UPSTAIRS         0        0         0        3                  15   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           5  
WALKING                            1  
WALKING_DOWNSTAIRS                10  
WALKING_UPSTAIRS                 450  


In [79]:
from sklearn.metrics import accuracy_score

In [80]:
accuracy_score(y_test, y_pred)

0.9538043478260869

# LSTM Model

In [125]:
# Create function returning a compiled network
def create_network(n_hidden, drop):
    
    # Initiliazing the sequential model
    model = Sequential()
    
    # Configuring the parameters
    model.add(LSTM(units=n_hidden, dropout=drop, return_sequences=True, input_shape=(timesteps, input_dim)))
    model.add(LSTM(units=n_hidden, dropout=drop, return_sequences=False))

    # Adding a dropout layer
    model.add(Dropout(drop))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    # model.summary()

    return model

In [126]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, verbose=0, epochs=epochs, batch_size=batch_size)

In [132]:
# Create grid search
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters, verbose=10, cv=2)

# Fit grid search
grid_result = grid.fit(X_train, Y_train)
grid_result.best_params_

In [33]:
# Initializing parameters
epochs = 20
batch_size = 50
# n_hidden = 32

In [66]:
n_hidden = 264
drop= 0.7
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(units = n_hidden, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(Dropout(drop))
model.add(LSTM(units =  n_hidden, return_sequences=False))
# Adding a dropout layer
model.add(Dropout(drop))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128, 264)          289344    
_________________________________________________________________
dropout_13 (Dropout)         (None, 128, 264)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 264)               558624    
_________________________________________________________________
dropout_14 (Dropout)         (None, 264)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 1590      
Total params: 849,558
Trainable params: 849,558
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [42]:
# Training the model
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/20
7352/7352 [==============================] - 227s 31ms/step - loss: 1.3779 - accuracy: 0.4022
Epoch 2/20
7352/7352 [==============================] - 229s 31ms/step - loss: 1.2833 - accuracy: 0.4256
Epoch 3/20
7352/7352 [==============================] - 227s 31ms/step - loss: 1.0420 - accuracy: 0.5279
Epoch 4/20
7352/7352 [==============================] - 225s 31ms/step - loss: 0.8007 - accuracy: 0.6323
Epoch 5/20
7352/7352 [==============================] - 222s 30ms/step - loss: 0.6743 - accuracy: 0.6783
Epoch 6/20
7352/7352 [==============================] - 225s 31ms/step - loss: 0.5519 - accuracy: 0.7628
Epoch 7/20
7352/7352 [==============================] - 228s 31ms/step - loss: 0.4023 - accuracy: 0.8564
Epoch 8/20
7352/7352 [==============================] - 223s 30ms/step - loss: 0.2483 - accuracy: 0.9125
Epoch 9/20
7352/7352 [==============================] - 225s 31ms/step - loss: 0.2317 - accuracy: 0.9279
Epoch 10/20
7352/7352 [==============================] 

In [43]:
# Confusion Matrix
print(confusion_matrix(Y_test, model1.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 534        0         0        0                   0   
SITTING                  5      381       103        0                   0   
STANDING                 0       96       436        0                   0   
WALKING                  0        0         0      472                  24   
WALKING_DOWNSTAIRS       0        0         0        6                 401   
WALKING_UPSTAIRS         0        2         0       24                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             3  
SITTING                            2  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                13  
WALKING_UPSTAIRS                 440  


In [44]:
score = model1.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [45]:
score

[0.4655049461999594, 0.9039701223373413]

# Observations and conclusion

In [103]:
from prettytable import PrettyTable    
x = PrettyTable()
x.field_names = ["Architecture", "Test Accuracy"]
x.add_row(["DNN Fusion", "0.95"])
x.add_row(["LSTM", "0.90"])

print(x)

+--------------+---------------+
| Architecture | Test Accuracy |
+--------------+---------------+
|  DNN Fusion  |      0.95     |
|     LSTM     |      0.90     |
+--------------+---------------+


- With a simple 2 layer architecture we got 90.39% accuracy and a loss of 0.46.
- Accuracy of DNN fusion based model is far better than the other nueral network architecture.
- From confusion matrix we can see that DNN fusion model is less confuse between sitting and standing label.